In [1]:
# dependencies 
%pip install mediapipe opencv-python scikit-learn pandas tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
#imports 
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib

Setting up data

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)
data_path1 = 'Public/asl_alphabet_train'
data_path = 'Public/train'
classifications = os.listdir(data_path)
classifications.sort()
print(classifications)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [6]:
img = cv2.imread(r"Public/asl_alphabet_train/A/A1.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print(hands.process(img).multi_hand_landmarks)
scaler = StandardScaler()

[landmark {
  x: 0.457428038
  y: 0.583321
  z: -6.47400668e-007
}
landmark {
  x: 0.570786357
  y: 0.503428459
  z: -0.0355797932
}
landmark {
  x: 0.640131116
  y: 0.373975933
  z: -0.0444065407
}
landmark {
  x: 0.651733875
  y: 0.264574826
  z: -0.0550685
}
landmark {
  x: 0.634490609
  y: 0.179893538
  z: -0.0590976737
}
landmark {
  x: 0.545696855
  y: 0.270556241
  z: -0.00423305947
}
landmark {
  x: 0.586574137
  y: 0.209667951
  z: -0.0716262534
}
landmark {
  x: 0.59239167
  y: 0.307403505
  z: -0.111358762
}
landmark {
  x: 0.589894652
  y: 0.393817753
  z: -0.122083895
}
landmark {
  x: 0.467299551
  y: 0.282378018
  z: -0.00464758603
}
landmark {
  x: 0.517436743
  y: 0.234142944
  z: -0.077506274
}
landmark {
  x: 0.530205071
  y: 0.349560529
  z: -0.0962087959
}
landmark {
  x: 0.534969032
  y: 0.43843478
  z: -0.0864398256
}
landmark {
  x: 0.39551264
  y: 0.304352134
  z: -0.0160673223
}
landmark {
  x: 0.45714274
  y: 0.292900413
  z: -0.0848888382
}
landmark {
  x: 0

Extracting media pip

In [101]:
x = []
y = []

count = 0

for label in tqdm(classifications):
    folder = os.path.join(data_path, label)
    count = 0
    for file in os.listdir(folder):
        if label == 'del' or label == 'nothing' or label == 'space':
            break
        if count > 50:
            break
        count += 1
        if file.endswith('.jpg'):
            img_path = os.path.join(folder, file)
            img= cv2.imread(img_path)

            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img)

            if results.multi_hand_landmarks:
                hand = results.multi_hand_landmarks[0]
                landmark_list = []
                for landmark in hand.landmark:
                    landmark_list.extend([landmark.x, landmark.y, landmark.z])
                if len(landmark_list) == 63:
                    x.append(landmark_list)
                    y.append(label.lower())

import pandas as pd



100%|██████████| 29/29 [00:50<00:00,  1.74s/it]


In [ ]:
x = []
y = []

test_path = 'Public/train'
test_files = sorted(os.listdir(test_path))  # this is the actual list of filenames

for file in tqdm(test_files):
    if file.endswith('.jpg'):
        img_path = os.path.join(test_path, file)
        img = cv2.imread(img_path)

        if img is None:
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img)

        if results.multi_hand_landmarks:
            hand = results.multi_hand_landmarks[0]
            landmark_list = []
            for landmark in hand.landmark:
                landmark_list.extend([landmark.x, landmark.y, landmark.z])
            if len(landmark_list) == 63:
                x.append(landmark_list)
                y.append(file[0])  # label like 'A', 'SPACE'



100%|██████████| 1508/1508 [01:10<00:00, 21.45it/s]


In [103]:
df = pd.DataFrame(x)
df['label'] = y
df.to_csv('processed_landmarks_combined_lite.csv', index=False)
print("Saved to processed_landmarks.csv")

Saved to processed_landmarks.csv


In [ ]:
# x_test = []
# y_test = []

# test_path = 'Public/asl_alphabet_test'
# test_files = sorted(os.listdir(test_path))  # this is the actual list of filenames

# for file in tqdm(test_files):
#     if file.endswith('.jpg'):
#         img_path = os.path.join(test_path, file)
#         img = cv2.imread(img_path)

#         if img is None:
#             continue

#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         results = hands.process(img)

#         if results.multi_hand_landmarks:
#             hand = results.multi_hand_landmarks[0]
#             landmark_list = []
#             for landmark in hand.landmark:
#                 landmark_list.extend([landmark.x, landmark.y, landmark.z])
#             if len(landmark_list) == 63:
#                 x_test.append(landmark_list)
#                 y_test.append(file.split('_')[0])  # label like 'A', 'SPACE'



100%|██████████| 28/28 [00:00<00:00, 30.27it/s]


In [132]:
x_test = []
y_test = []

test_path = 'Public/test'
test_files = sorted(os.listdir(test_path))  # this is the actual list of filenames

for file in tqdm(test_files):
    if file.endswith('.jpg'):
        img_path = os.path.join(test_path, file)
        img = cv2.imread(img_path)

        if img is None:
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img)

        if results.multi_hand_landmarks:
            hand = results.multi_hand_landmarks[0]
            landmark_list = []
            for landmark in hand.landmark:
                landmark_list.extend([landmark.x, landmark.y, landmark.z])
            if len(landmark_list) == 63:
                x_test.append(landmark_list)
                y_test.append(file[0])  # label like 'A', 'SPACE'

100%|██████████| 72/72 [00:02<00:00, 24.01it/s]


In [ ]:


# test_path = 'Public/asl_alphabet_test'
# test_files = sorted(os.listdir(test_path))  # this is the actual list of filenames

# for file in tqdm(test_files):
#     if file.endswith('.jpg'):
#         img_path = os.path.join(test_path, file)
#         img = cv2.imread(img_path)

#         if img is None:
#             continue

#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         results = hands.process(img)

#         if results.multi_hand_landmarks:
#             hand = results.multi_hand_landmarks[0]
#             landmark_list = []
#             for landmark in hand.landmark:
#                 landmark_list.extend([landmark.x, landmark.y, landmark.z])
#             if len(landmark_list) == 63:
#                 name = file.split('_')[0]
#                 if name== "nothing" or name == "space" or name == "del":
#                     break

#                 x_test.append(landmark_list)
#                 y_test.append(name.lower)  # label like 'A', 'SPACE'



 96%|█████████▋| 27/28 [00:01<00:00, 21.75it/s]


In [133]:
print(len(x_test), len(y_test))
le = LabelEncoder()
y_train = le.fit_transform(y)
y_test = le.transform(y_test)

# scaler = StandardScaler()
x_train = scaler.fit_transform(x)   
x_test = scaler.transform(x_test)
# x_train = x
# x_test = x_test
print(y_test)
print(y_train)


61 61
[ 0  1  1  1  2  2  2  3  5  5  6  6  6  6  7  7  8  8  9  9  9  9 10 10
 10 12 12 13 14 14 14 15 16 16 17 17 18 18 18 19 19 19 19 19 20 20 21 21
 21 21 22 23 23 23 23 24 24 25 25 25 25]
[26 26 26 ... 25 25 25]


In [134]:


#train
clf = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=1000, random_state=42)
clf.fit(x_train, y_train)

#save model
joblib.dump(clf, 'asl_model.pkl')
#save scaler
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.9, random_state=42)
#train
clf = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
clf.fit(x_train, y_train)
#save model
joblib.dump(clf, 'asl_model.pkl')
#save scaler
joblib.dump(scaler, 'scaler.pkl')


c:\Users\Kaito\Documents\asl-cv-service\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


['scaler.pkl']

In [135]:
acc = clf.score(x_test, y_test)
predictions = clf.predict(x_test)
print(y_test)
print(predictions)
print(f"Accuracy: {acc:.2%}")
with open('class_names.txt', 'w') as f:
    f.write('\n'.join(classifications))

[ 0  1  1  1  2  2  2  3  5  5  6  6  6  6  7  7  8  8  9  9  9  9 10 10
 10 12 12 13 14 14 14 15 16 16 17 17 18 18 18 19 19 19 19 19 20 20 21 21
 21 21 22 23 23 23 23 24 24 25 25 25 25]
[ 0 48  1  1  2  1  2  3  5  5  6  6  6  6  7  7  8  4  9  9  9  9 20 10
 36 13 12  4 14 41  4 15 16  9 17  2 19 18 18 18 19 19 13 19 20 20 17 36
 21 21 22 25 23 23 23 24 24 25 25 25 25]
Accuracy: 72.13%


In [53]:
import collections
print(collections.Counter(y_train))

Counter({'O': 406, 'Z': 404, 'Y': 404, 'K': 403, 'P': 402, 'S': 395, 'D': 392, 'R': 392, 'C': 390, 'F': 387, 'H': 382, 'Q': 380, 'G': 379, 'I': 374, 'E': 368, 'U': 367, 'T': 366, 'A': 358, 'B': 355, 'X': 352, 'J': 352, 'V': 351, 'L': 351, 'W': 333, 'space': 311, 'del': 311, 'M': 222, 'N': 134})
